In [55]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [65]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 3) #input -? OUtput? RF
        self.bn1 = nn.BatchNorm2d(10)
        self.conv2 = nn.Conv2d(10, 18, 3)
        self.bn2 = nn.BatchNorm2d(18)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout2d(0.15)
        self.conv3 = nn.Conv2d(18, 18, 3)
        self.bn3 = nn.BatchNorm2d(18)
        self.conv4 = nn.Conv2d(18, 18, 3)
        self.bn4 = nn.BatchNorm2d(18)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.dropout2 = nn.Dropout2d(0.15)
        self.conv5 = nn.Conv2d(18, 18, 3, padding=1)
        self.conv6 = nn.Conv2d(18, 18, 3, padding=1)
        #self.conv7 = nn.Conv2d(1024, 10, 3)
        self.fc1 = nn.Linear(4*4*18, 32)
        self.out = nn.Linear(32, 10)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool1(F.relu(self.bn2(self.conv2(x))))
        x = self.dropout1(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool2(F.relu(self.bn4(self.conv4(x))))
        x = self.dropout2(x)
        x = F.relu(self.conv5(x))
        #x = F.relu(self.conv6(x))
        x = x.reshape (-1, 4*4*18)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.out(x)
        return x
        #x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        #x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        #x = F.relu(self.conv6(F.relu(self.conv5(x))))
        #x = F.relu(self.conv7(x))
        #x = x.view(-1, 10)
        #return F.log_softmax(x)

In [66]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
print(use_cuda)
#print(torch.cuda.device_count())
#print(torch.cuda.memory_allocated(0)/1024)
#use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")
#device = torch.device("cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

True
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]             100
       BatchNorm2d-2           [-1, 10, 26, 26]              20
            Conv2d-3           [-1, 18, 24, 24]           1,638
       BatchNorm2d-4           [-1, 18, 24, 24]              36
         MaxPool2d-5           [-1, 18, 12, 12]               0
         Dropout2d-6           [-1, 18, 12, 12]               0
            Conv2d-7           [-1, 18, 10, 10]           2,934
       BatchNorm2d-8           [-1, 18, 10, 10]              36
            Conv2d-9             [-1, 18, 8, 8]           2,934
      BatchNorm2d-10             [-1, 18, 8, 8]              36
        MaxPool2d-11             [-1, 18, 4, 4]               0
        Dropout2d-12             [-1, 18, 4, 4]               0
           Conv2d-13             [-1, 18, 4, 4]           2,934
           Linear-14              

In [70]:


#torch.manual_seed(1)
#batch_size_train = 5000
#batch_size_test = 1000

#kwargs = {'num_workers': 4, 'pin_memory': True} if use_cuda else {}
#train_loader = torch.utils.data.DataLoader(
    #datasets.MNIST('../data', train=True, download=True,
                    #transform=transforms.Compose([
                        #transforms.ToTensor(),
                        #transforms.Normalize((0.1307,), (0.3081,))
                    #])),
    #batch_size=batch_size_train, shuffle=True, **kwargs)
#test_loader = torch.utils.data.DataLoader(
    #datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        #transforms.ToTensor(),
                        #transforms.Normalize((0.1307,), (0.3081,))
                    #])),
    #batch_size=batch_size_test, shuffle=True, **kwargs)
    
torch.manual_seed(1)
batch_size_train = 5000
batch_size_test = 1000

kwargs = {'num_workers': 4, 'pin_memory': True} if use_cuda else {}

# Define augmentation for training data
train_transforms = transforms.Compose([
    # Spatial augmentations
    transforms.RandomRotation((-10, 10)),              # Random rotation ±8 degrees
    transforms.RandomAffine(
        degrees=0,                                   # No additional rotation
        translate=(0.1, 0.1),                        # Random shift up to 10%
        scale=(0.9, 1.1),                           # Random scaling ±10%
        shear=(-5, 5)
        #fillcolor=0                                  # Fill empty space with black
    ),
    
    # Convert to tensor and normalize
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Keep test transforms simple
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Create data loaders with respective transforms
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        '../data', 
        train=True, 
        download=True,
        transform=train_transforms  # Apply augmentation only to training data
    ),
    batch_size=batch_size_train, 
    shuffle=True, 
    **kwargs
)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        '../data', 
        train=False, 
        transform=test_transforms  # No augmentation for test data
    ),
    batch_size=batch_size_test, 
    shuffle=True, 
    **kwargs
)

In [71]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        #loss = F.nll_loss(output, target)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    criterion = nn.CrossEntropyLoss()
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [72]:

model = Net().to(device)
optimizer = optim.AdamW(model.parameters(), lr=0.009, weight_decay=0.00001)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=1.3856781721115112 batch_id=11: 100%|██████████| 12/12 [00:12<00:00,  1.02s/it]



Test set: Average loss: 0.0007, Accuracy: 8191/10000 (81.91%)



loss=0.9842872619628906 batch_id=11: 100%|██████████| 12/12 [00:11<00:00,  1.02it/s]



Test set: Average loss: 0.0005, Accuracy: 8785/10000 (87.85%)



loss=0.8454082608222961 batch_id=11: 100%|██████████| 12/12 [00:13<00:00,  1.12s/it]


...


Test set: Average loss: 0.0001, Accuracy: 9903/10000 (99.03%)



loss=0.6048063039779663 batch_id=11: 100%|██████████| 12/12 [00:15<00:00,  1.26s/it]



Test set: Average loss: 0.0001, Accuracy: 9904/10000 (99.04%)



loss=0.597832441329956 batch_id=11: 100%|██████████| 12/12 [00:13<00:00,  1.15s/it] 



Test set: Average loss: 0.0001, Accuracy: 9907/10000 (99.07%)

